In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import sys
import os
import torch
import time
import random
import numpy as np
import wandb
from tqdm import tqdm
from early_stopping_pytorch import EarlyStopping
from models.resnet import resnet18, resnet34, resnet50

wandb.login(key="ef091b9abcea3186341ddf8995d62bde62d7469e")
wandb.init(project="PBL-2", name="resnet18")


"""
# 학습 재현성 고정
def fix_seed(seed, deterministic=False):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    if deterministic:
        torch.backends.cudnn.deterministic = True
    else:
        torch.backends.cudnn.benchmark = True  # 성능 향상을 위해 True로 변경

deterministic=True와 benchmark=False는 확실히 학습 속도를 저하시킬 수 있습니다.
특히 torch.backends.cudnn.benchmark=False는 CUDA가 최적의 알고리즘을 찾기 위한 
벤치마킹을 수행하지 않게 만들어 성능이 떨어질 수 있습니다.
# 속도 우선 설정 -> 완벽한 재현성은 보장되지 않음 

fix_seed(2025, deterministic=False)
"""

# WandB 설정
config = {
    "model": "resnet18",
    "batch_size": 128,
    "num_epochs": 100,
    "learning_rate": 0.001,
    "optimizer": "Adam",
    "seed": 2025,
    "deterministic": False,
}
wandb.config.update(config)

# CIFAR-100 데이터셋 로드
transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
])

trainset = torchvision.datasets.CIFAR100(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = DataLoader(trainset, batch_size=config["batch_size"], shuffle=True, num_workers=16)

testset = torchvision.datasets.CIFAR100(
    root='./data', train=False, download=True, transform=transform_test)
testloader = DataLoader(testset, batch_size=config["batch_size"], shuffle=False, num_workers=16)

def train(model, trainloader, criterion, optimizer, device, epoch):
    """
    학습 함수
    """
    model.train()   # 모델을 학습 모드로 설정
    start_time = time.time()  # 시간 측정 시작
    running_loss = 0.0
    correct_top1 = 0
    correct_top5 = 0
    total = 0
    
    for i, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        # top-1 정확도 계산
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct_top1 += predicted.eq(labels).sum().item()
        
        # top-5 정확도 계산
        _, top5_idx = outputs.topk(5, 1, largest=True, sorted=True)
        correct_top5 += top5_idx.eq(labels.view(-1, 1).expand_as(top5_idx)).sum().item()
        
        if (i + 1) % 50 == 0:  # 50 배치마다 출력
            print(f'Epoch [{epoch+1}], Batch [{i+1}/{len(trainloader)}], Loss: {loss.item():.4f}')
    
    epoch_loss = running_loss / len(trainloader)
    accuracy_top1 = 100.0 * correct_top1 / total
    accuracy_top5 = 100.0 * correct_top5 / total
    
    train_time = time.time() - start_time
    
    # 학습 세트에 대한 성능 출력
    print(f'Train set: Epoch: {epoch+1}, Average loss:{epoch_loss:.4f}, LR: {optimizer.param_groups[0]["lr"]:.6f} '
          f'Top-1 Accuracy: {accuracy_top1:.4f}%, Top-5 Accuracy: {accuracy_top5:.4f}%, Time consumed:{train_time:.2f}s')
    
    return epoch_loss, accuracy_top1, accuracy_top5

def evaluate(model, testloader, criterion, device, epoch):
    """
    평가 함수
    """
    model.eval()  # 모델을 평가 모드로 설정
    start_time = time.time()  # 시간 측정 시작
    
    test_loss = 0.0
    correct_top1 = 0
    correct_top5 = 0
    total = 0
    
    # 그래디언트 계산 비활성화
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # 순전파
            outputs = model(inputs)
            
            # 손실 계산
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            
            # top-1 정확도 계산
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct_top1 += (predicted == labels).sum().item()
            
            # top-5 정확도 계산
            _, top5_idx = outputs.topk(5, 1, largest=True, sorted=True)
            correct_top5 += top5_idx.eq(labels.view(-1, 1).expand_as(top5_idx)).sum().item()
    
    # 평균 손실 및 정확도 계산
    test_loss = test_loss / len(testloader)
    accuracy_top1 = 100.0 * correct_top1 / total
    accuracy_top5 = 100.0 * correct_top5 / total
    # 평가 시간 계산
    eval_time = time.time() - start_time
    
    # 테스트 세트에 대한 성능 출력
    print(f'Test set: Epoch: {epoch+1}, Average loss:{test_loss:.4f}, '
          f'Top-1 Accuracy: {accuracy_top1:.4f}%, Top-5 Accuracy: {accuracy_top5:.4f}%, Time consumed:{eval_time:.2f}s')
    print()
    
    return test_loss, accuracy_top1, accuracy_top5


# 메인 학습 루프
def main_training_loop(model, trainloader, testloader, criterion, optimizer, device, num_epochs):

    early_stopping = EarlyStopping(patience=patience, verbose=True)
    
    """
    메인 학습 루프
    """
    best_acc_top1 = 0.0
    best_acc_top5 = 0.0
    
    # tqdm을 사용한 진행 상황 표시
    for epoch in tqdm(range(num_epochs)):
        # 학습
        train_loss, train_acc_top1, train_acc_top5 = train(model, trainloader, criterion, optimizer, device, epoch)
        
        # 평가
        test_loss, test_acc_top1, test_acc_top5 = evaluate(model, testloader, criterion, device, epoch)
        
        # WandB에 로깅
        wandb.log({
            "epoch": epoch + 1,
            "learning_rate": optimizer.param_groups[0]['lr'],
            "train_loss": train_loss,
            "train_accuracy_top1": train_acc_top1,
            "train_accuracy_top5": train_acc_top5,
            "test_loss": test_loss,
            "test_accuracy_top1": test_acc_top1,
            "test_accuracy_top5": test_acc_top5
        })
            
        # 최고 정확도 모델 저장 (top-1 기준)
        if test_acc_top1 > best_acc_top1:
            best_acc_top1 = test_acc_top1
            best_acc_top5_at_best_top1 = test_acc_top5
            print(f'New best top-1 accuracy: {best_acc_top1:.2f}%, top-5 accuracy: {best_acc_top5_at_best_top1:.2f}%')
            # 모델 저장
            model_path = f'best_model_{wandb.run.name}.pth'
            torch.save(model.state_dict(), model_path)
            
            # WandB에 모델 아티팩트 저장
            wandb.save(model_path)
        
        # top-5 accuracy 기록 업데이트
        if test_acc_top5 > best_acc_top5:
            best_acc_top5 = test_acc_top5
            print(f'New best top-5 accuracy: {best_acc_top5:.2f}%')

            early_stopping(test_loss, model)
            if early_stopping.early_stop:
                print("Early stopping triggered. Training stopped.")
                break
    
        # 조기 종료된 경우 저장된 체크포인트에서 최적의 모델 로드
        model.load_state_dict(torch.load('checkpoint.pt'))
        
        print(f'Finish! Best top-1 accuracy: {best_acc_top1:.2f}%, Best top-5 accuracy: {best_acc_top5:.2f}%')
        wandb.run.summary["best_accuracy_top1"] = best_acc_top1
        wandb.run.summary["best_accuracy_top5"] = best_acc_top5

        if early_stopping.early_stop:
            wandb.run.summary["early_stopped"] = True
            wandb.run.summary["early_stopped_epoch"] = epoch+1



# 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 모델 초기화
model = resnet18().to(device)  
criterion = nn.CrossEntropyLoss()  # 손실 함수 정의
optimizer = optim.Adam(model.parameters(), lr=config["learning_rate"])  # 옵티마이저 정의

# WandB에 모델 구조 기록
wandb.watch(model, log="all")

# GPU 가속
if torch.cuda.device_count() > 1:
    print(f"{torch.cuda.device_count()}개의 GPU를 사용합니다.")
    model = nn.DataParallel(model)

# 훈련 시작 시간 기록
start_time = time.time()

# 메인 학습 루프 호출
main_training_loop(
    model=model,
    trainloader=trainloader,
    testloader=testloader,
    criterion=criterion,
    optimizer=optimizer,
    device=device,
    num_epochs=config["num_epochs"]
)

# 훈련 종료 시간 기록 및 출력
end_time = time.time()
total_time = end_time - start_time
wandb.log({"total_training_time": total_time})

print(f"Total training time: {total_time:.2f} seconds")

# WandB 실행 종료
wandb.finish()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


epoch,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇███
learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy_top1,▁▄▅▇▇█▇▇██▇████▇██████▇████████▇███▇████
test_accuracy_top5,▁▃▇▇██████▇█▇▇▇▇▇▇▇▇▇█▇▇▇▇▇▇█▇▇▇██▇▇▇██▇
test_loss,▇▄▃▁▁▁▂▃▃▃▄▄▄▅▅▆▅▅▆▆▆▇▆▇▇▆▇▇▇▇▇▇▇▇▇▇▇▇█▇
train_accuracy_top1,▁▂▃▄▅▆▇▇████████████████████████████████
train_accuracy_top5,▁▄▅▆▇███████████████████████████████████
train_loss,█▆▅▄▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,54
learning_rate,0.001
test_accuracy_top1,54.61


Files already downloaded and verified
Files already downloaded and verified
Using device: cuda
2개의 GPU를 사용합니다.


  0%|                                                                                                       | 0/100 [00:00<?, ?it/s]

Epoch [1], Batch [50/391], Loss: 4.1613
Epoch [1], Batch [100/391], Loss: 3.8398
Epoch [1], Batch [150/391], Loss: 3.5961
Epoch [1], Batch [200/391], Loss: 3.6111
Epoch [1], Batch [250/391], Loss: 3.4813
Epoch [1], Batch [300/391], Loss: 3.6301
Epoch [1], Batch [350/391], Loss: 2.7313
Train set: Epoch: 1, Average loss:3.6494, LR: 0.001000 Top-1 Accuracy: 13.8080%, Top-5 Accuracy: 37.4680%, Time consumed:142.21s


...

 99%|██████████████████████████████████████████████████████████████████████████████████████████ | 99/100 [4:29:15<02:42, 162.44s/it]

Test set: Epoch: 99, Average loss:3.8835, Top-1 Accuracy: 55.1200%, Top-5 Accuracy: 81.3200%, Time consumed:25.05s

Epoch [100], Batch [50/391], Loss: 0.0020
Epoch [100], Batch [100/391], Loss: 0.0048
Epoch [100], Batch [150/391], Loss: 0.0005
Epoch [100], Batch [200/391], Loss: 0.0045
Epoch [100], Batch [250/391], Loss: 0.0097
Epoch [100], Batch [300/391], Loss: 0.0260
Epoch [100], Batch [350/391], Loss: 0.0492
Train set: Epoch: 100, Average loss:0.0145, LR: 0.001000 Top-1 Accuracy: 99.5540%, Top-5 Accuracy: 99.9940%, Time consumed:138.81s


100%|██████████████████████████████████████████████████████████████████████████████████████████| 100/100 [4:31:58<00:00, 163.19s/it]

Test set: Epoch: 100, Average loss:3.7850, Top-1 Accuracy: 55.4000%, Top-5 Accuracy: 80.9300%, Time consumed:24.71s

Finish! Best top-1 accuracy: 56.49%, Best top-5 accuracy: 82.95%
Total training time: 16318.85 seconds


epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇████
learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy_top1,▁▃▆▇▇▇▇██▇██████████████████████████████
test_accuracy_top5,▁▇███████████████▇█████▇██▇█████████████
test_loss,▅▂▁▁▁▃▃▄▄▅▅▆▅▅▆▆▆▆▆▇▇▆▆▇▇▇▇▇▇▇█▇▇▇▇▇█▇██
total_training_time,▁
train_accuracy_top1,▁▅▇▇████████████████████████████████████
train_accuracy_top5,▁▄▅▆████████████████████████████████████
train_loss,█▇▄▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
best_accuracy_top1,56.49
best_accuracy_top5,82.95
